In [1]:
import pandas as pd
import openai
from dotenv import load_dotenv, find_dotenv
import os
import json
from tqdm import tqdm


In [2]:
cypher_rag = pd.read_csv('../data/results/cypher_rag_mcq_output.csv')


In [3]:
llm_for_evaluation = 'gpt-4'


In [4]:
load_dotenv(os.path.join(os.path.expanduser('~'), '.gpt_config.env'))
api_key = os.environ.get('API_KEY')
api_version = os.environ.get('API_VERSION')
resource_endpoint = os.environ.get('RESOURCE_ENDPOINT')
openai.api_type = 'azure'
openai.api_key = api_key
if resource_endpoint:
    openai.api_base = resource_endpoint
if api_version:
    openai.api_version = api_version

In [5]:
def fetch_GPT_response(instruction, system_prompt, chat_model_id, chat_deployment_id, temperature=0):
    # print('Calling OpenAI...')
    response = openai.ChatCompletion.create(
        temperature=temperature,
        deployment_id=chat_deployment_id,
        model=chat_model_id,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": instruction}
        ]
    )
    if 'choices' in response \
       and isinstance(response['choices'], list) \
       and len(response) >= 0 \
       and 'message' in response['choices'][0] \
       and 'content' in response['choices'][0]['message']:
        return response['choices'][0]['message']['content']
    else:
        return 'Unexpected response'

def get_GPT_response(instruction, system_prompt, chat_model_id, chat_deployment_id, temperature=0):
    return fetch_GPT_response(instruction, system_prompt, chat_model_id, chat_deployment_id, temperature)
    

In [6]:
system_prompt = '''
You are an expert evaluator of mcq question. You will be provided with an MCQ question which has 5 options out of which one option is correct.
You will also be provided with the correct label.
You will then provided with an answer.
You need to verify if that answer is correct or not, by checking it with the question and the label.
If the answer is correct, you return True, if the answer is not correct, you return False.
Return your answer in JSON format as follows:
{"response": <"True"/"False">}

Example 1:
Question: Out of the given list, which Gene is associated with psoriasis and Takayasu's arteritis. Given list is: SHTN1, HLA-B,  SLC14A2,  BTBD9,  DTNB
Label: HLA-B
Answer: The gene HLA-B is associated with both psoriasis and Takayasu's arteritis.
Your response: {"response": "True", "reason": "Answer is in agreement with the label."}

Example 2:
Question: Out of the given list, which Gene is associated with psoriasis and myelodysplastic syndrome. Given list is:  NOD2,  CHEK2, HLA-B, GCKR,  PKNOX2
Label: HLA-B
Answer: The genes associated with psoriasis and myelodysplastic syndrome from the given list are HLA-B and CHEK2.
Your response: {"response": "False", "reason":"Answer returns two options HLA-B and CHEK2, but the label says the correct answer is only HLA-B"}
'''

In [7]:
%%time

cypher_rag_eval = []
for index, row in tqdm(cypher_rag.iterrows()):
    question = row.text
    label = row.label
    answer = row.cypher_rag_answer
    prompt = f'''
    Question: {question}
    Label: {label}
    Answer: {answer}
    '''
    out = json.loads(get_GPT_response(prompt, system_prompt, llm_for_evaluation, llm_for_evaluation))
    cypher_rag_eval.append(out["response"])
    

306it [11:29,  2.25s/it]

CPU times: user 4.33 s, sys: 473 ms, total: 4.81 s
Wall time: 11min 29s


In [15]:
cypher_rag['cypher_rag_eval'] = cypher_rag_eval

In [18]:
cypher_rag.to_csv('../data/results/cypher_rag_mcq_output_with_eval.csv', index=False)
